In [5]:
from sympy.physics.quantum import TensorProduct,Dagger
from sympy import Symbol
from sympy import sin, cos, exp, sqrt, pi
from sympy.matrices import Matrix

theta = Symbol('theta')
phi = Symbol('phi')
p = Symbol('p')

def general_state_matrix():
    return Matrix([[cos(theta/2)**2, exp(1j*phi)*cos(theta/2)*sin(theta/2)],[exp(-1j*phi)*cos(theta/2)*sin(theta/2),sin(theta/2)**2]])
matriz_estado_geral = general_state_matrix()
#matriz_estado_geral

def Kbpf(j, p):
    if j == 0:
        return Matrix([[0,sqrt(p)],[0,0]])
    elif j == 1:
        return Matrix([[1,0],[0,sqrt(1-p)]])

#k0 = Kad(2,p)
#k0

In [11]:
import sympy as sym
from sympy.physics.quantum import TensorProduct, Dagger
from sympy.matrices import Matrix
theta = sym.Symbol('theta',real=True, positive=True)
phi = sym.Symbol('phi',real=True, positive=True)

p = sym.Symbol('p',real=True, positive=True)
gamma = sym.Symbol('gamma',real=True, positive=True)

Psi_AB = sym.Matrix([[
           sym.sqrt(p)*sym.cos(theta/2)],[
           sym.sqrt(p*gamma)*sym.exp(1j*phi)*sym.sin(theta/2)],[
           sym.sqrt((1-p)*(1-gamma))*sym.cos(theta/2)],[
           0],[
           sym.sqrt(p*(1-gamma))*sym.exp(1j*phi)*sym.sin(theta/2)],[
           0],[
           sym.sqrt(1-p)*sym.exp(1j*phi)*sym.sin(theta/2)],[
           sym.sqrt(1-p)*gamma*sym.cos(theta/2)
           ]]) 
# Psi_AB = sym.Array([[sym.sqrt(p)*sym.cos(theta/2),
#            sym.sqrt(p*gamma)*sym.exp(1j*phi)*sym.sin(theta/2),
#            sym.sqrt((1-p)*(1-gamma))*sym.cos(theta/2),
#            0,
#            sym.sqrt(p*(1-gamma))*sym.exp(1j*phi)*sym.sin(theta/2),
#            0,
#            sym.sqrt(1-p)*sym.exp(1j*phi)*sym.sin(theta/2),
#            sym.sqrt(1-p)*gamma*sym.cos(theta/2)
#            ]]) 
Psi_AB.shape
result = Psi_AB*Dagger(Psi_AB)
result.simplify()
result



Matrix([
[                                p*cos(theta/2)**2,                                 sqrt(gamma)*p*exp(-1.0*I*phi)*sin(theta)/2,                             sqrt(p)*cos(theta/2)**2*conjugate(sqrt((gamma - 1)*(p - 1))), 0,                                 p*exp(-1.0*I*phi)*sin(theta)*conjugate(sqrt(1 - gamma))/2, 0,                   sqrt(p)*exp(-1.0*I*phi)*sin(theta)*conjugate(sqrt(1 - p))/2,                             gamma*sqrt(p)*cos(theta/2)**2*conjugate(sqrt(1 - p))],
[        sqrt(gamma)*p*exp(1.0*I*phi)*sin(theta)/2,                                                    gamma*p*sin(theta/2)**2,     sqrt(gamma)*sqrt(p)*exp(1.0*I*phi)*sin(theta)*conjugate(sqrt((gamma - 1)*(p - 1)))/2, 0,                                  sqrt(gamma)*p*sin(theta/2)**2*conjugate(sqrt(1 - gamma)), 0,                    sqrt(gamma)*sqrt(p)*sin(theta/2)**2*conjugate(sqrt(1 - p)),          gamma**(3/2)*sqrt(p)*exp(1.0*I*phi)*sin(theta)*conjugate(sqrt(1 - p))/2],
[sqrt(p)*sqrt((gamma - 1)*(p - 1))*co

In [10]:
def trace(A):
    d = A.shape[0]; tr = 0.0
    for j in range(0,d):
       tr += A[j,j]
    return tr
trace(result).subs(theta,pi).subs(gamma,0.5).subs(p,1)

1.00000000000000

In [ ]:
from sympy.physics.quantum import TensorProduct
from sympy import conjugate

rho_AB = TensorProduct(conjugate(Psi_AB),Psi_AB)
rho_AB

Matrix([
[                                       cos(conjugate(theta)/2)*conjugate(sqrt(p))],
[      exp(-1.0*I*conjugate(phi))*sin(conjugate(theta)/2)*conjugate(sqrt(gamma*p))],
[                     cos(conjugate(theta)/2)*conjugate(sqrt((1 - gamma)*(1 - p)))],
[                                                                                0],
[exp(-1.0*I*conjugate(phi))*sin(conjugate(theta)/2)*conjugate(sqrt(p*(1 - gamma)))],
[                                                                                0],
[        exp(-1.0*I*conjugate(phi))*sin(conjugate(theta)/2)*conjugate(sqrt(1 - p))],
[                  cos(conjugate(theta)/2)*conjugate(gamma)*conjugate(sqrt(1 - p))]])xMatrix([
[                           sqrt(p)*cos(theta/2)],
[      sqrt(gamma*p)*exp(1.0*I*phi)*sin(theta/2)],
[         sqrt((1 - gamma)*(1 - p))*cos(theta/2)],
[                                              0],
[sqrt(p*(1 - gamma))*exp(1.0*I*phi)*sin(theta/2)],
[                                             

In [ ]:
from sympy.vector import CoordSys3D
N = CoordSys3D('N')
v1 = 2*N.i+3*N.j-N.k
v2 = N.i-4*N.j+N.k
v1.dot(v2)
v3 = v1.cross(v2)
v3

(-1)*N.i + (-3)*N.j + (-11)*N.k

In [ ]:
target_op = state_generalized_amplitude_damping(np.pi/2, 0, p,1)
alpha = eval(input("How many rows? "))
beta = eval(input("How many columns? "))

def make_matrix(alpha,beta):
    matrix_thing = SHIT.random.randint(0,50,(alpha,beta))
    return(matrix_thing)

matrix_sympy = Matrix(make_matrix(alpha, beta))